python 3.7 and pytorch 1.8

# pytorch 搭建简单前馈和BP神经网络模型

- 目的：搭建简单前馈和BP神经网络模型
- 主要步骤：
1. 建立并训练前馈神经网络模型(无反向传播更新权重)
2. 建立并训练BP神经网络模型


## 0. 初始化全局变量

In [ ]:
EPOCHS = 10 # 全部训练集的训练次数
INPUT_DIM = 128  # 输入数据的大小为 128
HIDDEN_DIM = 216  # 隐层大小为 216，或者叫latent_dim
OUTPUT_DIM = 3  # 输出大小为 3


## 1. 建立并训练前馈神经网络模型(无反向传播更新权重)

In [ ]:
import torch.nn as nn
# F 相当于tf中的layers，具有多种层类型
import torch.nn.functional as F

class Net(nn.Module):
    # 模型定义，继承 Module 类
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Net, self).__init__()
        # 隐藏层，全连接层，参数大小：input_dim * hidden_dim
        # Linear层 == Dense层(tf)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # 输出层，全连接层，参数大小：hidden_dim * output_dim
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)  # 输入 x 经过隐藏层线性运算
        x = F.relu(x)  # 经过激活函数 ReLU 运算
        x = F.relu(self.fc2(x))  # 经过输出层线性及激活运算，最后输出
        return x

In [3]:
# 模型初始化
my_first_model = Net(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(my_first_model)  # 打印显示模型结构

Net(
  (fc1): Linear(in_features=128, out_features=216, bias=True)
  (fc2): Linear(in_features=216, out_features=3, bias=True)
)


In [4]:
import torch
# 生成数据
N = 10  # 假设一批数据为 10 个
x = torch.randn(N, INPUT_DIM)  # 数据大小: [10，128]

In [7]:
# 训练模型
# 不需要调用 forward(),基类nn.Module实现了__call__方法
output = my_first_model(x)
print(output)

tensor([[0.0000e+00, 3.3696e-01, 5.3513e-01],
        [0.0000e+00, 2.0388e-01, 1.8619e-01],
        [0.0000e+00, 6.6536e-01, 0.0000e+00],
        [0.0000e+00, 6.3716e-04, 0.0000e+00],
        [0.0000e+00, 2.4092e-01, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 8.3801e-02],
        [0.0000e+00, 7.4178e-02, 8.0430e-03],
        [0.0000e+00, 5.0477e-01, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 9.4443e-02, 8.5941e-02]], grad_fn=<ReluBackward0>)


## 2. 建立并训练BP神经网络模型

In [8]:
# 随机产生一百对输入输出数据
x = torch.randn(100, INPUT_DIM)
y = torch.randint(0, 3, (100,))  # 输出值为 0,1,2 三种类别

(tensor([[ 0.2936,  0.4346,  0.8534,  ..., -0.7977,  0.2747, -0.1186],
         [ 1.0822,  0.1648,  0.9615,  ..., -2.0863,  0.4900, -0.2376],
         [ 0.6113, -0.1821,  0.5124,  ...,  0.0608,  0.6543,  0.4685],
         ...,
         [ 1.7240, -1.3237,  0.4254,  ..., -1.4699, -0.4850,  1.2870],
         [-0.0077,  0.8927,  0.2616,  ...,  0.2258, -1.4090,  1.5387],
         [-0.0249, -1.6181, -1.1035,  ..., -0.1918, -0.9758,  0.5093]]),
 tensor([2, 2, 2, 1, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 2, 2, 1, 2, 1, 0, 0, 0, 1, 0,
         2, 2, 0, 2, 1, 2, 1, 0, 1, 2, 0, 2, 1, 0, 2, 2, 2, 0, 2, 2, 1, 0, 1, 2,
         2, 0, 2, 2, 2, 0, 2, 1, 2, 2, 1, 2, 1, 2, 0, 2, 2, 1, 2, 1, 1, 0, 0, 2,
         1, 2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 2, 1, 0, 1,
         0, 2, 2, 1]))

In [ ]:
# 定义损失函数为分类所用的交叉熵
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# 定义优化器：Adam为带一阶速度和二阶加速度的梯度下降方法、
optimizer = torch.optim.Adam(my_first_model.parameters(), lr=0.01)

In [ ]:
for t in range(EPOCHS):  # 应用数据反复训练 10 次，也称为 10 个 epoch
    # 第一步：数据的前向传播，计算预测值 p_pred
    y_pred = my_first_model(x)
    # 第二步：计算计算预测值 p_pred 与真实值的误差
    loss = criterion(y_pred, y)
    print(f"第 {t} 个 Epoch, 损失是 {loss.item()}")
    # 在反向传播之前，将模型的梯度归零(tf不用显式归零梯度)
    my_first_model.zero_grad()
    # 第三步：反向传播误差
    loss.backward()
    # 直接通过梯度一步到位，更新整个网络的训练参数
    optimizer.step()
